## 패션 스타일별 소득 금액 예측 모델 구현

### 1. 이미지 데이터 변환 / json 파일에서 필요한 라벨 추출

In [2]:
import pandas as pd
import json
import os
import cv2
import numpy as np


# (1) json 파일에서 이미지에 대한 라벨을 추출하고 이미지 크기를 조절하는 전처리 함수 구현
def read_img(folder_path):

    file_list = os.listdir(folder_path)
    feature_list = []
    label_df = pd.DataFrame(columns=['era','style','age','job','income'])

    for file in file_list:
        # json 파일 로드, 필요한 정보 추출
        with open(folder_path+file, 'r') as f: 
            data = json.load(f)
        
        # json 파일에서 파일명 읽어와 이미지 데이터 로드/300*400/0~1 정규화 => array 저장
        img_path = folder_path.replace('label','img') + data['imgName']
        img = cv2.imread(img_path)
        img = cv2.resize(img, (300,400)) / 255.
        
        feature_list.append(img)

        # 라벨 데이터 정수형으로 변경
        k = ["feminine","classic","minimal","popart","space","hippie","disco",
            "military","punk","powersuit","bodyconscious","hiphop","kitsch",
            "lingerie","grunge","cityglam","oriental","ecology","sportivecasual",
            "athleisure","lounge","normcore","genderless"]
        v = range(0,23)
        style_dict = dict(zip(k, v))

        # 라벨 df 생성
        labels = [data['item']['era'], style_dict[data['item']['style']], data['user']['age'], data['user']['job'], data['user']['income']]
        label_df.loc[label_df.shape[0]] = labels

    # 최종 feature 데이터 생성
    feature = np.array(feature_list)

    return feature, label_df

### 2. 데이터 로딩, 범주별 데이터 현황 파악

In [6]:
import pandas as pd
import os

# style 컬럼 딕셔너리
k = ["feminine","classic","minimal","popart","space","hippie","disco",
    "military","punk","powersuit","bodyconscious","hiphop","kitsch",
    "lingerie","grunge","cityglam","oriental","ecology","sportivecasual",
    "athleisure","lounge","normcore","genderless"]
v = range(0,23)
style_dict = dict(zip(k, v))


# 타겟 데이터를 하나로 병합
df = pd.DataFrame(columns=['era','style','age','job','income'])
path = './data/target/'

for file in os.listdir(path):
    one_df = pd.read_csv(path+file)
    df = pd.concat([df, one_df], ignore_index=True)

    # 각 파일별 최소 개수를 기준으로 데이터를 가져올 생각, 추출하여 확인
    print(one_df['income'].value_counts()[5])

# 최소 개수에 맞추어 다운샘플링하기 위한 개수 파악
df['income'].value_counts()

223
167
174
208
145
162
201
160


income
2    7113
1    6744
3    3332
4    2039
6    1828
5    1440
Name: count, dtype: int64

### 3) 데이터 통합

In [110]:
import numpy as np

# 모델별 데이터를 1000개로 축소하여 npy/npz 파일로 변환하였음
# npy 데이터
feature_1990 = (np.load('./data/feature/1990.npy')*255).astype('uint8')
feature_2000 = (np.load('./data/feature/2000.npy')*255).astype('uint8')
feature_2019 = (np.load('./data/feature/2019.npy')*255).astype('uint8')

# npz 데이터
feature_1950 = (np.load('./data/feature/1950.npz')['images']*255).astype('uint8')
feature_1960 = (np.load('./data/feature/1960.npz')['images']*255).astype('uint8')
feature_1970 = (np.load('./data/feature/1970.npz')['images']*255).astype('uint8')
feature_1980 = (np.load('./data/feature/1980.npz')['images']*255).astype('uint8')
feature_2010 = (np.load('./data/feature/2010.npz')['images']*255).astype('uint8')

# 피처 데이터를 하나로 병합
feature = np.concatenate((feature_1950, feature_1960, feature_1970, feature_1980, feature_1990, feature_2000, feature_2010, feature_2019), axis=0)

# 통합된 데이터 저장
np.save('all.npy', feature)

# float16 형식으로 데이터 로드
feature_16 = np.load('./all.npy').astype('float16')

feature_16 = feature_16 / 255.

#  float16 형식으로 데이터 저장
np.save('all_16.npy', feature_16)

### 4. 모델링

In [ ]:
from sklearn.model_selection import train_test_split

# 피처 데이터 로딩
feature = np.load('./all_16.npy')

# 타겟 데이터를 하나로 병합
df = pd.DataFrame(columns=['era','style','age','job','income'])
path = './data/target/'

for file in os.listdir(path):
    one_df = pd.read_csv(path+file)
    df = pd.concat([df, one_df], ignore_index=True)

# 라벨 원핫인코딩
label = pd.get_dummies(df['income'], dtype=int)

# 훈련/테스트용 데이터 분리
train_X, test_X, train_y, test_y = train_test_split(feature, label, shuffle=True, stratify=label, random_state=777)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Activation, BatchNormalization, ReLU
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

# 모델 구현
model = Sequential()

# 컨볼루션 레이어 추가
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(100,75,3)))
# model.add(BatchNormalization())
model.add(Activation(ReLU()))
# model.add(MaxPooling2D())
  
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))     
# model.add(BatchNormalization())
model.add(Activation(ReLU()))                    
model.add(MaxPooling2D())

# model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))                         
# # model.add(BatchNormalization())
# model.add(Activation(ReLU()))                    
# # model.add(MaxPooling2D())

# model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))     
# # model.add(BatchNormalization())
# model.add(Activation(ReLU()))                    
# model.add(MaxPooling2D())

# model.add(Dropout(0.25)) 

model.add(Flatten())                       

model.add(Dense(32, kernel_initializer='he_normal', activation='relu'))
model.add(Dense(6, kernel_initializer='he_normal', activation='softmax'))

# 에포크 별 최고 성능 모델 저장
checkpoint = ModelCheckpoint('./best_model4.hdf5', monitor='val_loss', verbose=True, save_best_only=True)

# 얼리스타핑
early_stop = EarlyStopping(monitor='val_loss', patience=30)

# # 러닝레이트
# def scheduler(epoch, lr):
#     if epoch > 20:
#         return 0.000001
#     else:
#         return lr

# learning_rate = LearningRateScheduler(scheduler)


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

model.fit(feature, label, epochs=1000, batch_size=32, validation_split=0.2, callbacks=[checkpoint], verbose=1)

### 5. 모델 테스트

In [ ]:
from keras.models import load_model

# 모델 로드
model = load_model('./cnn_fashion_model_INCOME.hdf5')

# 성능 평가
model.predict(test_X)